In [1]:
import argparse
import os
import logging
import tensorflow as tf
import numpy as np
import pickle

In [2]:
# Specify the file path of the saved model
file_path = '/Users/pratikhotchandani/Downloads/Github/DeepLearning/Transfer Learning/artifacts/model.pkl'

# Load the saved model from the file
with open(file_path, 'rb') as file:
    loaded_model = pickle.load(file)

Metal device set to: Apple M1 Pro


2023-07-12 13:14:09.390891: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-12 13:14:09.391059: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
loaded_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [4]:
# freeze weights
for layer in loaded_model.layers[: -1]:
    print(f"before freezing weights {layer.name}: {layer.trainable}") 
    layer.trainable = False
    print(f"after freezing weights {layer.name}: {layer.trainable}") 

# modify last layer for our problem statement
base_layers = loaded_model.layers[:-1]

new_model = tf.keras.models.Sequential(base_layers)
new_model.add(
    tf.keras.layers.Dense(2, activation="softmax", name="output_layer")
)

new_model.summary()

before freezing weights flatten: True
after freezing weights flatten: False
before freezing weights dense: True
after freezing weights dense: False
before freezing weights leaky_re_lu: True
after freezing weights leaky_re_lu: False
before freezing weights dense_1: True
after freezing weights dense_1: False
before freezing weights leaky_re_lu_1: True
after freezing weights leaky_re_lu_1: False
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               3

In [5]:
def update_odd_even_labels(labels):
    # odd = 0
    # even = 1
    for idx, label in enumerate(labels):
        labels[idx] = np.where(label%2 == 0, 1, 0)
    return labels

In [6]:
# get the data
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [7]:
y_train_bin, y_test_bin, y_valid_bin = update_odd_even_labels([y_train, y_test, y_valid])


In [8]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

history = new_model.fit(X_train, y_train_bin, epochs=10,
                validation_data=(X_valid, y_valid_bin), verbose=2)

new_model.evaluate(X_test, y_test_bin)

Epoch 1/10


2023-07-12 13:31:43.846527: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-12 13:31:43.978199: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-07-12 13:31:51.072428: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1719/1719 - 8s - loss: 0.4348 - accuracy: 0.7931 - val_loss: 0.3629 - val_accuracy: 0.8400 - 8s/epoch - 5ms/step
Epoch 2/10
1719/1719 - 7s - loss: 0.3624 - accuracy: 0.8390 - val_loss: 0.3362 - val_accuracy: 0.8562 - 7s/epoch - 4ms/step
Epoch 3/10
1719/1719 - 7s - loss: 0.3428 - accuracy: 0.8501 - val_loss: 0.3207 - val_accuracy: 0.8636 - 7s/epoch - 4ms/step
Epoch 4/10
1719/1719 - 7s - loss: 0.3302 - accuracy: 0.8569 - val_loss: 0.3107 - val_accuracy: 0.8700 - 7s/epoch - 4ms/step
Epoch 5/10
1719/1719 - 7s - loss: 0.3211 - accuracy: 0.8614 - val_loss: 0.3024 - val_accuracy: 0.8734 - 7s/epoch - 4ms/step
Epoch 6/10
1719/1719 - 7s - loss: 0.3138 - accuracy: 0.8653 - val_loss: 0.2963 - val_accuracy: 0.8776 - 7s/epoch - 4ms/step
Epoch 7/10
1719/1719 - 7s - loss: 0.3078 - accuracy: 0.8681 - val_loss: 0.2914 - val_accuracy: 0.8798 - 7s/epoch - 4ms/step
Epoch 8/10
1719/1719 - 7s - loss: 0.3027 - accuracy: 0.8706 - val_loss: 0.2856 - val_accuracy: 0.8820 - 7s/epoch - 4ms/step
Epoch 9/10
1719/171

[0.2837076783180237, 0.8831000328063965]

In [9]:
# Specify the file path including the directory where you want to store the model
file_path = '/Users/pratikhotchandani/Downloads/Github/DeepLearning/Transfer Learning/transfer_learning_model.pkl'

# Save the model to the specified file path
with open(file_path, 'wb') as file:
    pickle.dump(new_model, file)

INFO:tensorflow:Assets written to: ram://4434844e-4540-4668-bbe9-cf259810e0dd/assets
